In [4]:
# import the libraries
from pandas import read_csv
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot
import numpy
import pandas as pd
import tensorflow as tf
tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [5]:
# read the data
df = pd.read_csv('../data/airbnb_total.csv', index_col=0)

In [6]:
df['satisfaction'] = df['satisfaction'].apply(lambda x: 1 if x >= 95 else 0)

In [7]:
df.shape

(43486, 117)

In [8]:
#df.isnull().values.any()

In [9]:
# split data into X and y
array = df.values
X = array[:,0:116]
y = array[:,116]

In [10]:
# grid search
model = XGBClassifier(
 learning_rate =1,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
n_estimators = [100, 1000]
max_depth = [1, 100]
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators)
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="roc_auc", n_jobs=-1, cv=kfold, verbose=1)

In [11]:
# fit the model
grid_result = grid_search.fit(X, y)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:  4.7min remaining:   56.1s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  4.9min finished


In [12]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))
# plot results
scores = numpy.array(means).reshape(len(max_depth), len(n_estimators))
for i, value in enumerate(max_depth):
    pyplot.plot(n_estimators, scores[i], label='depth: ' + str(value))
pyplot.legend()
pyplot.xlabel('n_estimators')
pyplot.ylabel('Log Loss')
pyplot.savefig('n_estimators_vs_max_depth.png')

Best: 0.937508 using {'max_depth': 1, 'n_estimators': 100}
0.937508 (0.001051) with: {'max_depth': 1, 'n_estimators': 100}
0.934523 (0.000674) with: {'max_depth': 1, 'n_estimators': 1000}
0.920165 (0.002285) with: {'max_depth': 100, 'n_estimators': 100}
0.922078 (0.001991) with: {'max_depth': 100, 'n_estimators': 1000}
